In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import time
import sys

from sklearn.utils import shuffle
from tensorflow.python.platform import gfile
from tensorflow.python.framework import graph_util
from PIL import Image

In [2]:
w = 60
h = 80
BATCH_SIZE = 5
DATA_SIZE = w*h
LAYER_H = 64
OUT_NUM = 10
STDDEV = 0.1

In [3]:
def load4csv(filename, size=2048, k=255.0):

    dataframe = pd.read_csv(filename)
    feature_cols = dataframe.columns[:-1] # all but image column #去除最后一列数据，最后一列为image
    # transform image space-separated pixel values to normalized pixel vector

    dataframe['data'] = dataframe['data'].apply(lambda img: np.fromstring(img, sep = ' ') / k)
    dataframe = dataframe.dropna() # drop entries w/NaN entries
    X = np.vstack(dataframe['data'])
    X = X.reshape(-1, size, 1)
    
    y = dataframe[feature_cols].values
    
    X, y = shuffle(X, y)
    return X, y

In [4]:
train_data, train_labels = load4csv('data.csv', w*h, 255.0)

In [5]:
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)#平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)#标准差
        tf.summary.scalar('max', tf.reduce_max(var))#最大值
        tf.summary.scalar('min', tf.reduce_min(var))#最小值
        tf.summary.histogram('histogram', var)#直方图

with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, [None, DATA_SIZE], name='x')
    variable_summaries(x)
    y = tf.placeholder(tf.float32, [None, OUT_NUM], name='y')
    variable_summaries(y)
    k = tf.placeholder(tf.float32, name='k')
    variable_summaries(k)
    
with tf.name_scope('layer'):
    with tf.name_scope('h1'):
        with tf.name_scope('v'):
            h1w = tf.Variable(tf.truncated_normal([DATA_SIZE, LAYER_H], stddev=STDDEV), name='h1w')
            variable_summaries(h1w)
            h1b = tf.Variable(tf.constant(STDDEV, shape=[LAYER_H]), name='h1b')
            variable_summaries(h1b)
        with tf.name_scope('p'):
            dense = tf.reshape(x, [-1, DATA_SIZE], name='dense')
            variable_summaries(dense)
            h1 = tf.nn.relu(tf.matmul(dense, h1w) + h1b)
            variable_summaries(h1)
            h1drop = tf.nn.dropout(h1, k)
            variable_summaries(h1drop)
    with tf.name_scope('out'):
        with tf.name_scope('v'):
            low = tf.Variable(tf.truncated_normal([LAYER_H, OUT_NUM], stddev=STDDEV), name='low')
            variable_summaries(low)
            lob = tf.Variable(tf.constant(STDDEV, shape=[OUT_NUM]), name='lob')
            variable_summaries(lob)
        with tf.name_scope('p'):
            y_out = tf.nn.softmax(tf.add(tf.matmul(h1drop, low), lob), name='output')

In [6]:
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.reduce_sum(tf.square(y-y_out), 1))
    tf.summary.scalar('loss', loss)
with tf.name_scope('train'):
#     train_step = tf.train.AdamOptimizer(learning_rate, 0.95).minimize(loss, global_step=global_step)
    train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_out, 1))
    with tf.name_scope('correct_prediction'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar('accuracy', accuracy)
init = tf.global_variables_initializer()
merged = tf.summary.merge_all()
print("loss")

loss


In [7]:
with tf.Session() as sess:
    #执行初始化
    sess.run(init)
    writer = tf.summary.FileWriter('logs/', sess.graph)
    saver = tf.train.Saver(max_to_keep=2)
    for epoch in range(1000):
        for i in range(8):
            offset = i * BATCH_SIZE
            batch_xs = train_data[offset:(offset + BATCH_SIZE), ...]
#             print(batch_xs.shape)
            batch_xs = batch_xs.reshape(-1, DATA_SIZE)
            batch_ys = train_labels[offset:(offset + BATCH_SIZE)]
            
            summary,_ = sess.run([merged,train_step],feed_dict={x:batch_xs, y:batch_ys, k: 0.8})
            _y = sess.run(y_out, feed_dict={x:batch_xs, y:batch_ys, k: 1.0})
            
#             print(i, _y.shape[0], _y[0][0])

        if epoch % 10 == 0:
            print(epoch)
            train_accuacy = accuracy.eval(feed_dict={x:batch_xs, y:batch_ys, k: 1.0})
            _y = sess.run(y_out, feed_dict={x:batch_xs, y:batch_ys, k: 1.0})
            _loss = sess.run(loss, feed_dict={x:batch_xs, y:batch_ys, k: 1.0})
            print("%s %s %s %s %s %s %s %s %s %s"%(_y[0][0],_y[0][1],_y[0][2],_y[0][3],_y[0][4],_y[0][5],_y[0][6],_y[0][7],_y[0][8],_y[0][9]))
            print("%s %s %s %s %s %s %s %s %s %s"%(batch_ys[0][0],batch_ys[0][1],batch_ys[0][2],batch_ys[0][3],batch_ys[0][4],batch_ys[0][5],batch_ys[0][6],batch_ys[0][7],batch_ys[0][8],batch_ys[0][9]))
            print("loss %f training accuracy %f"%(_loss, train_accuacy))
            print("-------------")
    print("---end---")
    graph = graph_util.convert_variables_to_constants(sess, sess.graph_def, ['layer/out/p/output'])
    tf.train.write_graph(graph, '.', 'num.pb', as_text=False)

0
0.389523 0.177723 0.0103468 0.0181601 0.144399 0.131836 0.000961292 0.0299836 0.00301592 0.094051
0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0
loss 1.182175 training accuracy 0.000000
-------------
10
0.167478 0.0974909 0.081637 0.12365 0.131275 0.195007 0.0336139 0.0933664 0.0172013 0.0592812
0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0
loss 0.900823 training accuracy 0.400000
-------------
20
0.0635482 0.0317253 0.0137053 0.0385633 0.0371939 0.640821 0.0930483 0.0350277 0.00796145 0.0384052
0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0
loss 0.507926 training accuracy 0.600000
-------------
30
0.0251238 0.0353995 0.00712633 0.0203148 0.025374 0.70572 0.0981481 0.0253894 0.00795969 0.0494441
0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0
loss 0.235410 training accuracy 1.000000
-------------
40
0.0138387 0.0156179 0.0025536 0.0094443 0.00923056 0.854468 0.0603906 0.0146094 0.00441694 0.01543
0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0
loss 0.089390 training accuracy 1.000000
-------------
50
0.00657696 0.01436

400
4.04478e-05 0.000233673 1.80918e-05 9.47914e-05 0.000361456 0.997914 0.000376467 0.000164943 0.000640301 0.000155808
0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0
loss 0.000017 training accuracy 1.000000
-------------
410
1.80503e-05 0.000112435 8.9458e-06 2.15565e-05 0.00030982 0.998946 0.000232985 9.22336e-05 0.000116061 0.000141671
0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0
loss 0.000015 training accuracy 1.000000
-------------
420
4.20949e-05 0.000463988 1.85428e-05 9.10759e-05 0.000258686 0.997882 0.000484698 0.000133597 0.000408007 0.000216801
0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0
loss 0.000027 training accuracy 1.000000
-------------
430
1.40318e-05 0.000172879 1.4502e-05 9.81373e-05 0.00036395 0.998646 0.000261195 8.74361e-05 0.000154736 0.000187132
0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0
loss 0.000003 training accuracy 1.000000
-------------
440
1.53313e-05 0.000145261 6.9186e-06 6.06517e-05 0.000726255 0.998284 0.000188115 0.000187238 0.000234681 0.000151536
0.0 0.0 0.0 0.0 0.0 

790
1.00984e-06 4.18392e-05 2.70057e-06 4.0714e-06 0.000107208 0.9996 9.40749e-05 2.42825e-05 5.60564e-05 6.93687e-05
0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0
loss 0.000000 training accuracy 1.000000
-------------
800
6.18936e-07 1.3757e-05 1.13606e-06 2.04933e-06 6.68137e-05 0.999834 3.82116e-05 7.11548e-06 1.46461e-05 2.16774e-05
0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0
loss 0.000000 training accuracy 1.000000
-------------
810
1.42189e-06 1.37639e-05 7.69119e-07 3.54977e-06 9.40323e-05 0.999821 3.75605e-05 7.16721e-06 1.04719e-05 1.02597e-05
0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0
loss 0.000000 training accuracy 1.000000
-------------
820
8.44255e-07 1.15351e-05 7.17365e-07 2.07899e-06 8.85244e-05 0.999826 2.96687e-05 6.82803e-06 2.13957e-05 1.25096e-05
0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0
loss 0.000000 training accuracy 1.000000
-------------
830
5.17583e-07 7.56844e-06 7.38737e-07 5.20805e-07 3.03288e-05 0.999839 6.44522e-05 5.67359e-06 2.85765e-05 2.26936e-05
0.0 0.0 0.0 0.0 0.0

In [ ]:
print(len(train_data))
print(train_data.shape)